Tentando calcular as funções influência dos momentos que vou usar na estimação

1. Ler os dados do projeto dentro de Julia
2. Calcular os momentos usando os dados
3. Calcular IF
    * se a observação não pertence ao conjunto (i.e., o momento é sobre casos pré-corte e a observação fica dentro da corte), IF = 0
    * do contrário, IF = x - Momento(X), onde x é a observação e X é a amostra correspondente àquele momento

Estamos trocando alguns momentos

- 5 e 6 serão taxas de recuperação para S e J em casos "rápidos" reorganizados dentro da corte
    - entrarão no lugar da taxa de recuperação pre-court, pois temos só 2 observações para estes
    
- 9 e 10 serão as variâncias das taxas de recuperação de S e J para reorganizações dentro da corte
    - mais momentos para identificar habilidades iniciais de S e de J
    
- 11 será a variância da duração total de casos ~reorganizados~ dentro da corte
    - ajudará a identificar \rho, torcemos
    - teste no aguia foi com casos reorganizados, mas o MM3 é a média da duração de todos os casos dentro da corte. Portanto, vou calcular a variância de todos os casos dentro da corte no MM11
    
- 12 será o coeficiente da regressão da taxa de recuperação total na duração total para casos reorganizados dentro da corte
    - é o momento 9 de Dou et al, que serve para identificar \rho

In [1]:
#lendo dados do projeto: trilemma_data.xlsx
# using Pkg
# Pkg.add("XLSX")

using XLSX

In [2]:
using DataFrames

In [3]:
using Statistics

In [4]:
ENV["COLUMNS"] = 240 #para aumentar o número de colunas mostradas

240

In [115]:
df = DataFrame(XLSX.readtable("trilemma_data.xlsx", "Sheet1")...)

,processo,fil_date,data_start,plan_date,final_date,months_fil_start,months_start_plan,months_plan_final,months_fil_final,in_court,D_junior,D_labor,D_senior,R_junior,R_labor,R_senior,assets_total,assets_fixed
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0000547-55.2014.8.26.0291,2014-01-22,2014-01-29,2015-09-16,2018-07-12,0.233333,19.8333,34.3333,54.4,1.0,3.49201e6,1.38557e6,3.656e6,0.442533,0.941905,0.442533,2.53837e7,2.36164e7
2,0000731-65.2011.8.26.0695,2011-03-22,2011-04-08,2011-10-20,2017-04-20,0.566667,6.5,66.9667,74.0333,1.0,1.58332e6,9.64064e5,1.61192e5,0.81201,0.941905,0.81201,2.77487e6,1.0668e6
3,0003124-79.2014.8.26.0299,2014-05-28,2014-09-25,2016-07-18,2018-07-02,4.0,22.0667,23.8,49.8667,1.0,9.85081e6,1.63535e5,3.30049e6,0.36744,0.968244,0.36744,1.1296e7,3.32642e6
4,0006202-38.2010.8.26.0100,2010-02-12,2010-03-29,2011-01-03,2017-08-25,1.5,9.33333,80.8667,91.7,1.0,3.58351e7,1.72202e6,4.18984e5,0.535184,0.941905,0.535184,1.65361e7,1.15454e7
5,0011100-26.2012.8.26.0100,2012-03-06,2012-04-04,2014-05-15,2015-11-24,0.966667,25.7,18.6,45.2667,1.0,1.43452e7,0.0,1.41472e7,0.465931,0.0,0.465931,1.86502e7,5.63179e6
6,0014297-52.2013.8.26.0100,2013-02-07,2013-02-22,2014-07-21,2018-05-15,0.5,17.1333,46.4667,64.1,1.0,3.1438e7,1.41865e6,1.48989e7,0.396523,0.941905,0.536727,6.79508e7,1.23996e7
7,0016095-59.2013.8.26.0161,2013-06-17,2014-03-18,2016-06-06,2018-11-01,9.13333,27.0333,29.2667,65.4333,1.0,1.00002e7,75754.0,1.02842e5,0.814917,0.941905,0.814917,3.684e6,259066.0
8,0023189-81.2012.8.26.0100,2012-05-07,2012-06-22,2013-06-07,2019-01-15,1.53333,11.6667,68.2667,81.4667,1.0,4.86213e6,0.0,3.844e6,0.206332,0.0,0.206332,2.43727e6,755068.0
9,0025113-93.2013.8.26.0100,2013-04-03,2013-08-16,2014-09-17,2018-09-24,4.5,13.2333,48.9333,66.6667,1.0,1.07158e6,0.0,150000.0,0.448017,0.0,0.448017,299845.0,243594.0


In [117]:
names(df)

32-element Array{Symbol,1}:
 :processo
 :fil_date
 :data_start
 :plan_date
 :final_date
 :months_fil_start
 :months_start_plan
 :months_plan_final
 :months_fil_final
 :in_court
 :D_junior
 :D_labor
 :D_senior
 ⋮
 :Vh
 :L
 :D
 :Dj_D
 :L_D
 :Vh_D
 :Reorg_incourt
 :Reorg_precourt
 :Liq_incourt
 :Liq_precourt
 :tempo_medio_agc
 :ln_n_months

In [118]:
INCOURT = filter(df -> df.in_court .== 1.0, df);
PRECOURT = filter(df -> df.in_court .== 0.0, df);

In [119]:
filter(INCOURT -> INCOURT.months_start_plan .== 0, INCOURT)

,processo,fil_date,data_start,plan_date,final_date,months_fil_start,months_start_plan,months_plan_final,months_fil_final,in_court,D_junior,D_labor,D_senior,R_junior,R_labor,R_senior,assets_total,assets_fixed,outcome_bargain
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any


MM1: average log number of months between observed proposals, for incourt cases

In [120]:
mm1 = mean(skipmissing(df.ln_n_months))

2.1118291318473066

In [124]:
if1 = zeros(nrow(df));

for i in 1:length(if1)
    if(ismissing(df.ln_n_months[i]))
        if1[i] = zero(0.0)
    else
        if1[i] = df.ln_n_months[i] - mm1
    end
end

#testando se a média é maior que zero
#qual seria o zero em Julia?
mean(if1)> 0.000000000000001

false

In [125]:
if1

66-element Array{Float64,1}:
  0.18238771147622224
  0.0
  0.0
  0.12176308965978766
  0.4415146794939222
  0.036049571385757595
  1.185241540605951
 -1.9576784520200483
  0.47090976717772826
  0.0
  1.0280034856804412
 -0.6787307045296134
  0.0
  ⋮
  0.6935494132154658
  0.0
 -0.6636038129969257
  0.0
  0.1253281558242838
  0.0
  0.8430811471864295
  0.0
 -0.9162245771918873
  0.0
  0.0
  0.0

MM2: fraction reorganized given that the case went into court

In [126]:
mm2 = INCOURT
mm2 = size(filter(mm2 -> mm2.outcome_bargain .== "reorganized", mm2),1)/ size(mm2,1)

0.84375

In [127]:
if2 = zeros(nrow(df));

for i in 1:length(if2)
    if(df.in_court[i]==0.0)
        if2[i] = zero(0.0)
    else
        if2[i] = df.Reorg_incourt[i] - mm2
    end
end

mean(if2)

0.0

MM3: avg ln duration of court cases in months

In [128]:
mm3 = mean(log.(INCOURT.months_start_plan))

2.760695631787451

In [129]:
if3 = zeros(nrow(df));

for i in 1:length(if3)
    if(df.in_court[i]==0.0)
        if3[i] = zero(0.0)
    else
        if3[i] = log(df.months_start_plan[i]) - mm3
    end
end

mean(if3)

2.4895910249170175e-16

MM4: Fraction of cases incourt

In [130]:
mm4 = mean(df.in_court)

0.9696969696969697

In [131]:
#função influência do mm4:
if4 = zeros(nrow(df));

# for i in 1:length(if4)
#     println(i)
# end

for i in 1:length(if4)
    if4[i] = df.in_court[i] - mm4
end

In [132]:
mean(if4)

-2.6914497566670462e-17

MM5: avg recovery rate for S given REORGANIZATION in the 25% fastest cases

Includes PRECOURT and INCOURT cases

In [133]:
q = 0.25 #quartil desejado


mm5 = filter(df -> df.outcome_bargain .== "reorganized", df) #casos reorganizados, dentro ou fora da corte

,processo,fil_date,data_start,plan_date,final_date,months_fil_start,months_start_plan,months_plan_final,months_fil_final,in_court,D_junior,D_labor,D_senior,R_junior,R_labor,R_senior,assets_total,assets_fixed
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0000547-55.2014.8.26.0291,2014-01-22,2014-01-29,2015-09-16,2018-07-12,0.233333,19.8333,34.3333,54.4,1.0,3.49201e6,1.38557e6,3.656e6,0.442533,0.941905,0.442533,2.53837e7,2.36164e7
2,0000731-65.2011.8.26.0695,2011-03-22,2011-04-08,2011-10-20,2017-04-20,0.566667,6.5,66.9667,74.0333,1.0,1.58332e6,9.64064e5,1.61192e5,0.81201,0.941905,0.81201,2.77487e6,1.0668e6
3,0003124-79.2014.8.26.0299,2014-05-28,2014-09-25,2016-07-18,2018-07-02,4.0,22.0667,23.8,49.8667,1.0,9.85081e6,1.63535e5,3.30049e6,0.36744,0.968244,0.36744,1.1296e7,3.32642e6
4,0006202-38.2010.8.26.0100,2010-02-12,2010-03-29,2011-01-03,2017-08-25,1.5,9.33333,80.8667,91.7,1.0,3.58351e7,1.72202e6,4.18984e5,0.535184,0.941905,0.535184,1.65361e7,1.15454e7
5,0011100-26.2012.8.26.0100,2012-03-06,2012-04-04,2014-05-15,2015-11-24,0.966667,25.7,18.6,45.2667,1.0,1.43452e7,0.0,1.41472e7,0.465931,0.0,0.465931,1.86502e7,5.63179e6
6,0014297-52.2013.8.26.0100,2013-02-07,2013-02-22,2014-07-21,2018-05-15,0.5,17.1333,46.4667,64.1,1.0,3.1438e7,1.41865e6,1.48989e7,0.396523,0.941905,0.536727,6.79508e7,1.23996e7
7,0016095-59.2013.8.26.0161,2013-06-17,2014-03-18,2016-06-06,2018-11-01,9.13333,27.0333,29.2667,65.4333,1.0,1.00002e7,75754.0,1.02842e5,0.814917,0.941905,0.814917,3.684e6,259066.0
8,0023189-81.2012.8.26.0100,2012-05-07,2012-06-22,2013-06-07,2019-01-15,1.53333,11.6667,68.2667,81.4667,1.0,4.86213e6,0.0,3.844e6,0.206332,0.0,0.206332,2.43727e6,755068.0
9,0025113-93.2013.8.26.0100,2013-04-03,2013-08-16,2014-09-17,2018-09-24,4.5,13.2333,48.9333,66.6667,1.0,1.07158e6,0.0,150000.0,0.448017,0.0,0.448017,299845.0,243594.0


In [134]:
quartil = quantile(log.(mm5.months_start_plan), q)

2.348195604246101

In [135]:
mm5 = filter(mm5 -> log.(mm5.months_start_plan) .<= quartil, mm5)

,processo,fil_date,data_start,plan_date,final_date,months_fil_start,months_start_plan,months_plan_final,months_fil_final,in_court,D_junior,D_labor,D_senior,R_junior,R_labor,R_senior,assets_total,assets_fixed
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0000731-65.2011.8.26.0695,2011-03-22,2011-04-08,2011-10-20,2017-04-20,0.566667,6.5,66.9667,74.0333,1.0,1.58332e6,9.64064e5,1.61192e5,0.81201,0.941905,0.81201,2.77487e6,1.0668e6
2,0006202-38.2010.8.26.0100,2010-02-12,2010-03-29,2011-01-03,2017-08-25,1.5,9.33333,80.8667,91.7,1.0,3.58351e7,1.72202e6,4.18984e5,0.535184,0.941905,0.535184,1.65361e7,1.15454e7
3,0025535-05.2012.8.26.0100,2012-05-18,2012-09-03,2013-06-07,2015-10-08,3.6,9.23333,28.4333,41.2667,1.0,8.35837e6,3.05502e5,1.26003e6,0.726299,0.941905,0.726299,2.62484e7,812587.0
4,0133085-64.2009.8.26.0100,2009-04-01,2009-05-19,2010-03-19,2018-12-04,1.6,10.1333,106.067,117.8,1.0,1.66432e7,89600.0,1.12e6,0.377744,0.995025,0.377744,1.25236e7,1.60186e6
5,0199374-13.2008.8.26.0100,2008-09-23,2008-10-30,2009-09-09,2017-09-20,1.23333,10.4667,97.7667,109.467,1.0,6.89941e6,11642.0,1.16586e6,0.687773,0.941905,0.687773,8.34309e6,718390.0
6,0228439-53.2008.8.26.0100,2008-06-15,2009-01-09,2009-11-12,2016-04-19,6.93333,10.2333,78.3333,95.5,1.0,4.24744e7,9.57444e5,684573.0,0.598274,0.941905,0.598274,1.46139e7,6.41127e6
7,1001156-81.2014.8.26.0077,2014-02-25,2014-03-07,2015-01-15,2018-10-16,0.333333,10.4667,45.6667,56.4667,1.0,1.73876e6,86650.6,8.51832e5,0.173783,0.941746,0.173783,2.06973e6,576866.0
8,1006499-72.2016.8.26.0664,2016-08-09,2016-09-12,2017-06-07,2019-08-14,1.13333,8.93333,26.6,36.6667,1.0,1.67434e6,60166.6,4.18353e6,0.182776,0.953758,0.180101,1.12756e7,3.3634e6
9,1015481-89.2014.8.26.0100,2014-02-18,2014-03-20,2014-11-06,2017-05-16,1.0,7.7,30.7333,39.4333,1.0,6.36258e6,3.82754e5,1.26671e7,0.278199,0.941905,0.278199,2.69186e7,3.36751e6


In [136]:
mm5 = mean(mm5.R_senior)

0.5652848834410633

In [137]:
quartil

2.348195604246101

In [138]:
if5 = zeros(nrow(df));

for i in 1:length(if5)
    #casos liquidados ou que não estão entre os 25% reorganizados mais rapidamente não influenciam este momento
    if(df.outcome_bargain[i] == "liquidated")
        if5[i] = zero(0.0)
    elseif( (df.outcome_bargain[i] == "reorganized" && log(df.months_start_plan[i]) > quartil ) )
        if5[i] = zero(0.0)
    else
        if5[i] = df.R_senior[i] - mm5
    end
end

mean(if5)

5.046468293750712e-18

In [139]:
#se a IF estiver certa, somente 15 observações não terão valor zero
size(filter(if5 -> if5 .!= 0.0, if5), 1)

#bingo!

15

MM6: avg recovery arte for J given REORGANIZATION in the 25% fastest cases

Includes PRECOURT and INCOURT cases

In [140]:
q = 0.25 #quartil desejado

mm6 = filter(df -> df.outcome_bargain .== "reorganized", df) #casos reorganizados, dentro ou fora da corte

quartil = quantile(log.(mm6.months_start_plan), q)

mm6 = filter(mm6 -> log.(mm6.months_start_plan) .<= quartil, mm6)

mm6 = mean(mm6.R_junior)

0.5562758267330555

In [141]:
if6 = zeros(nrow(df));

for i in 1:length(if6)
    #casos liquidados ou que não estão entre os 25% reorganizados mais rapidamente não influenciam este momento
    if(df.outcome_bargain[i] == "liquidated")
        if6[i] = zero(0.0)
    elseif( (df.outcome_bargain[i] == "reorganized" && log(df.months_start_plan[i]) > quartil ) )
        if6[i] = zero(0.0)
    else
        if6[i] = df.R_junior[i] - mm6
    end
end



mean(if6)

3.027880976250427e-17

In [142]:
#se a IF estiver certa, somente 15 observações não terão valor zero
size(filter(if6 -> if6 .!= 0.0, if6), 1)
#bingo!

15

MM7: junior creditor’s average fraction gain, conditional on an in-court reorganization

In [143]:
INCOURT_REORG = filter(INCOURT -> INCOURT.outcome_bargain .== "reorganized", INCOURT);

In [145]:
INCOURT_REORG.payoff_j = INCOURT_REORG.R_junior .* INCOURT_REORG.D_junior;
INCOURT_REORG.payoff_s = INCOURT_REORG.R_senior .* INCOURT_REORG.D_senior;

In [146]:
mm7 = INCOURT_REORG
mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s ))

0.7733661464016862

In [147]:
df.payoff_j = df.R_junior .* df.D_junior;
df.payoff_s = df.R_senior .* df.D_senior;

In [148]:
if7 = zeros(nrow(df));

for i in 1:length(if7)
    if(df.Reorg_incourt[i]== 0.0) #zero se a observação não for de reorganização dentro da corte
        if7[i] = zero(0.0)
    else
        if7[i] = (df.payoff_j[i] / (df.payoff_j[i] + df.payoff_s[i]) ) - mm7
    end
end

mean(if7)

-3.1960965860421176e-17

MM8: avg total recovery rate given incourt REORGANIZATION

In [149]:
mm8 = INCOURT_REORG
mm8 = mean((mm8.payoff_s .+ mm8.payoff_j) ./ (mm8.D_senior .+ mm8.D_junior))

0.4977287552407836

In [150]:
if8 = zeros(nrow(df));

for i in 1:length(if8)
    if(df.Reorg_incourt[i]==0.0) #zero se a observação não for de reorganização dentro da corte
        if8[i] = zero(0.0)
    else
        if8[i] = ((df.payoff_s[i] + df.payoff_j[i]) / (df.D_senior[i] + df.D_junior[i] ) )  - mm8
    end
end

mean(if8)

-9.25185853854297e-18

In [151]:
if8

66-element Array{Float64,1}:
 -0.05519535561448358
  0.31428094257096345
 -0.13028884505016558
  0.03745484319769937
 -0.03179753420385556
 -0.056125618203346384
  0.3171884166107624
 -0.29139685705980456
 -0.049711984791891595
  0.22857064470807936
  0.0
 -0.113115061209212
 -0.0781546994540806
  ⋮
 -0.19455627843283635
  0.014516804047653376
  0.09909386280285243
  0.0
  0.23137185581361525
 -0.014851125108793661
 -0.1285290563205761
  0.0
 -0.14742536042306126
 -0.056253496253599544
  0.0
  0.0

MM9: variance of recovery rate of S given INCOURT REORG

In [152]:
mm9 = INCOURT_REORG
mm9 = var(mm9.R_senior)

0.04652980615833147

In [153]:
if9 = zeros(nrow(df));

for i in 1:length(if9)
    if(df.Reorg_incourt[i]==0.0) #zero se a observação não for de reorganização dentro da corte
        if9[i] = zero(0.0)
    else
        if9[i] = (df.R_senior[i] - mean(INCOURT_REORG.R_senior))^2  - mm9
    end
end

mean(if9)

-0.0007049970630050308

MM10: variance of recovery rate of J given INCOURT REORG

In [154]:
mm10 = INCOURT_REORG
mm10 = var(mm10.R_junior)

0.03647614100988236

In [155]:
if10 = zeros(nrow(df));

for i in 1:length(if10)
    if(df.Reorg_incourt[i]==0.0) #zero se a observação não for de reorganização dentro da corte
        if10[i] = zero(0.0)
    else
        if10[i] = (df.R_junior[i] - mean(INCOURT_REORG.R_junior))^2  - mm10
    end
end

mean(if10)

-0.0005526688031800269

MM11: variance ln duration of court cases in months

É a parte de variância, ao invés da média (MM3)

In [156]:
mm11 = var(log.(INCOURT.months_start_plan))

0.16966778593934911

In [157]:
filter(df -> df.in_court .== 0, df)

,processo,fil_date,data_start,plan_date,final_date,months_fil_start,months_start_plan,months_plan_final,months_fil_final,in_court,D_junior,D_labor,D_senior,R_junior,R_labor,R_senior,assets_total,assets_fixed
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1071904-64.2017.8.26.0100,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.66866e9,0.0,4.44703e8,0.805483,0.0,0.805483,2.07274e9,9.76e6
2,1125916-28.2017.8.26.0100,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.7173e5,0.0,3.30293e6,0.308326,0.0,0.441695,1.92569e6,25752.2


In [158]:
(4-2)^2

4

In [159]:
if11 = zeros(nrow(df));

for i in 1:length(if11)
    if(df.in_court[i]==0.0) #zero se a observação não for dentro da corte
        if11[i] = zero(0.0)
    else
        if11[i] = (log(df.months_start_plan[i]) - mean(log.(INCOURT.months_start_plan)) )^2  - mm11
    end
end

mean(if11)

-0.0025707240293840364

MM12: slope of regression of total recovery rate on total duration for incourt reorganizations

Igual ao MM9 de Dou et al

In [160]:
y = INCOURT_REORG

y = log.(y.months_start_plan)

x = INCOURT_REORG
x = log.( (x.payoff_s .+ x.payoff_j) ./ (x.D_senior .+ x.D_junior))

X = [ones(size(x,1)) x]

using LinearAlgebra
mm12 = (X' * X) \ (X' * y)

mm12 = mm12[2]

-0.09822867075174749

In [161]:
x[2]

-0.2082429957738625

In [162]:
if12 = zeros(nrow(df));

length(if12)

66

In [163]:
if12 = zeros(nrow(df));

#precisa recriar y e x aqui embaixo para englobar todas va
Y = log.(df.months_start_plan)
X = log.( (df.payoff_s .+ df.payoff_j) ./ (df.D_senior .+ df.D_junior))

66-element Array{Float64,1}:
 -0.815239338001109
 -0.2082429957738625
 -1.0011954833838048
 -0.6251454163057129
 -0.763717250090964
 -0.817343680245939
 -0.20466880053592001
 -1.5782692504939808
 -0.8029246132300827
 -0.3197929525456788
 -1.568541694581116
 -0.9555158405550106
 -0.8685152351945186
  ⋮
 -1.193453405033666
 -0.66895116093255
 -0.5161353319440475
 -1.208503993624127
 -0.31594354407601666
 -0.727992011193047
 -0.9964175918461682
 -0.9116146662622224
 -1.0489556576531676
 -0.8176332990424642
 -0.21631350816334852
 -0.8556012024942443

In [166]:
for i in 1:length(if12)
    if(df.Reorg_incourt[i]== 0.0) #zero se a observação não for de reorganização dentro da corte
        if12[i] = zero(0.0)
        
        #X e Y são as observações para toda a base de dados
        #x e y são as observações somente para os casos reorganizados dentro da corte
    else
        if12[i] = ( (X[i] - mean(x)) / var(x) ) * (  (Y[i] - mean(y)) - mm12 * (X[i] - mean(x)))
    end
end

mean(if12)

1.4903903027543768e-15

## Momentos novos


Lembrando que:
- months_start_plan = duração total da barganha
- ln_n_months = tempo entre AGCs, que tem vários valores faltando

MM10: taxa de recuperação de reorganizações dentro da corte 25% mais rápidas

OBS: a taxa de recuperação não é descrescente nos quartis. Nem sempre os casos mais rápidos são os que têm maior taxa de recuperação. 

In [165]:
q = 0.25 #quartil desejado

mm10 = INCOURT_REORG
quartil = quantile(log.(mm10.months_start_plan), q)

2.403082335814427

In [166]:
mm10 = filter(mm10 -> log.(mm10.months_start_plan) .<= quartil, mm10)

,processo,fil_date,data_start,plan_date,final_date,months_fil_start,months_start_plan,months_plan_final,months_fil_final,in_court,D_junior,D_labor,D_senior,R_junior,R_labor,R_senior,assets_total,assets_fixed
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0000731-65.2011.8.26.0695,2011-03-22,2011-04-08,2011-10-20,2017-04-20,0.566667,6.5,66.9667,74.0333,1.0,1.58332e6,9.64064e5,1.61192e5,0.81201,0.941905,0.81201,2.77487e6,1.0668e6
2,0006202-38.2010.8.26.0100,2010-02-12,2010-03-29,2011-01-03,2017-08-25,1.5,9.33333,80.8667,91.7,1.0,3.58351e7,1.72202e6,4.18984e5,0.535184,0.941905,0.535184,1.65361e7,1.15454e7
3,0025535-05.2012.8.26.0100,2012-05-18,2012-09-03,2013-06-07,2015-10-08,3.6,9.23333,28.4333,41.2667,1.0,8.35837e6,3.05502e5,1.26003e6,0.726299,0.941905,0.726299,2.62484e7,812587.0
4,0133085-64.2009.8.26.0100,2009-04-01,2009-05-19,2010-03-19,2018-12-04,1.6,10.1333,106.067,117.8,1.0,1.66432e7,89600.0,1.12e6,0.377744,0.995025,0.377744,1.25236e7,1.60186e6
5,0199374-13.2008.8.26.0100,2008-09-23,2008-10-30,2009-09-09,2017-09-20,1.23333,10.4667,97.7667,109.467,1.0,6.89941e6,11642.0,1.16586e6,0.687773,0.941905,0.687773,8.34309e6,718390.0
6,0228439-53.2008.8.26.0100,2008-06-15,2009-01-09,2009-11-12,2016-04-19,6.93333,10.2333,78.3333,95.5,1.0,4.24744e7,9.57444e5,684573.0,0.598274,0.941905,0.598274,1.46139e7,6.41127e6
7,1000926-42.2013.8.26.0152,2013-02-22,2013-04-15,2014-03-10,2016-02-05,1.73333,10.9667,23.2333,35.9333,1.0,9.653e6,33000.0,387000.0,0.167423,0.941905,0.167423,5.13303e6,108040.0
8,1001156-81.2014.8.26.0077,2014-02-25,2014-03-07,2015-01-15,2018-10-16,0.333333,10.4667,45.6667,56.4667,1.0,1.73876e6,86650.6,8.51832e5,0.173783,0.941746,0.173783,2.06973e6,576866.0
9,1006499-72.2016.8.26.0664,2016-08-09,2016-09-12,2017-06-07,2019-08-14,1.13333,8.93333,26.6,36.6667,1.0,1.67434e6,60166.6,4.18353e6,0.182776,0.953758,0.180101,1.12756e7,3.3634e6


In [167]:
mm10 = mean((mm10.payoff_s .+ mm10.payoff_j) ./ (mm10.D_senior .+ mm10.D_junior))

0.5274119591220352

MM11: taxa de recuperação de S em reorganizações dentro da corte 25% mais rápidas

In [171]:
q = 0.25 #quartil desejado

mm11 = INCOURT_REORG
quartil = quantile(log.(mm11.months_start_plan), q)

mm11 = filter(mm11 -> log.(mm11.months_start_plan) .<= quartil, mm11)
@show mm11_teste = mean(mm11.R_senior) #dá no mesmo
mm11 = mean((mm11.payoff_s ./ mm11.D_senior ))

mm11_teste = mean(mm11.R_senior) = 0.5285370654448899


0.5285370654448899

MM12: taxa de recuperação de J reorganizações dentro da corte 25% mais rápidas

In [172]:
q = 0.25 #quartil desejado

mm12 = INCOURT_REORG
quartil = quantile(log.(mm12.months_start_plan), q)

mm12 = filter(mm12 -> log.(mm12.months_start_plan) .<= quartil, mm12)
@show mm12_teste = mean(mm12.R_junior) #dá no mesmo
mm12 = mean((mm12.payoff_j ./ mm12.D_junior ))

mm12_teste = mean(mm12.R_junior) = 0.5284108439775381


0.528410843977538

MM13: variância do log da duração dos casos reorganizados dentro da corte

In [174]:
mm13 = var(log.(INCOURT_REORG.months_start_plan))

0.15666387636379916

MM14: tempo médio dos casos que foram liquidados dentro da corte

Como já tem a duração média de todos os casos dentro da corte (MM3), não precisa calcular a duração média das reorganizações dentro da corte porque elas se tornam redundantes.

In [177]:
INCOURT_LIQ = filter(INCOURT -> INCOURT.outcome_bargain .== "liquidated", INCOURT);

mm14 = mean(log.(INCOURT_LIQ.months_start_plan))

3.1496130110309557

MM15: fração do valor do ativo da firma que foi extraído pelos credores

A ideia é que isso possa identificar a chance de o Devedor ser sorteado a jogar, eventualmente

In [186]:
mm15 = INCOURT_REORG

mm15 = mean( ((mm15.payoff_s .+ mm15.payoff_j) ./ (mm15.D_senior .+ mm15.D_junior)) ./ mm15.Vh_D)

# mean( (mm15.payoff_s .+ mm15.payoff_j) ./ mm15.Vh ) #se dividir por Vh dá no mesmo, D só está normalizando tudo

0.6966074986282449

MM16: variância da taxa de recuperação dentro da corte

In [187]:
mm16 = INCOURT_REORG
mm16 = var((mm16.payoff_s .+ mm16.payoff_j) ./ (mm16.D_senior .+ mm16.D_junior))

0.03742054006325057

MM 17: variância da taxa de recuperação de S dentro da corte

In [190]:
mm17 = INCOURT_REORG
mm17 = var((mm17.R_senior))

0.04652980615833147

MM18: variância da taxa de recuperação de J dentro da corte

In [191]:
mm18 = INCOURT_REORG
mm18 = var((mm18.R_junior))

0.03647614100988236





### Parte 2: empilhando as funções influência

Seguindo o cálculo mostrado em Khan, página 6

Checar depois se entendi

IF = hcat(if1, if2, if3, if4, if5, if6, if7, if8)

In [36]:
n = nrow(df);

In [37]:
n^2

4356

In [38]:
W = 1/(n^2) * transpose(IF) * IF

8×8 Array{Float64,2}:
  0.00597747   -0.000437554   0.00114374   -2.03325e-19   0.0           0.0          -5.31024e-5   -4.01707e-5
 -0.000437554   0.00193698   -0.000892831  -1.1117e-20    0.0           0.0          -7.35587e-20  -1.60918e-20
  0.00114374   -0.000892831   0.00245387    1.12634e-19   0.0           0.0          -0.000158145  -0.000178559
 -2.0073e-19   -9.79382e-21   1.14328e-19   0.000445224  -1.02573e-20  -2.21669e-20  -1.18291e-20  -7.81986e-21
  0.0           0.0           0.0          -1.07859e-20   1.51907e-5    2.07598e-5    0.0           0.0
  0.0           0.0           0.0          -1.69448e-20   2.07598e-5    2.83706e-5    0.0           0.0
 -5.31024e-5   -6.34745e-20  -0.000158145  -1.10154e-20   0.0           0.0           0.000646001   0.000103483
 -4.01707e-5   -2.20758e-20  -0.000178559  -7.38802e-21   0.0           0.0           0.000103483   0.0004553

In [39]:
using LinearAlgebra

In [40]:
diag(W)

8-element Array{Float64,1}:
 0.005977465822316409
 0.001936983471074379
 0.0024538729371393476
 0.0004452235857194535
 1.5190712068603778e-5
 2.8370583805089904e-5
 0.0006460012814914279
 0.00045530041858408635

In [44]:
omega = inv(W)

8×8 Array{Float64,2}:
 183.829          1.63823      -86.5358           1.0552e-13      -2.53796e-13     1.85712e-13    -3.35731       -16.9554
   1.63823      625.232        235.595           -4.07709e-14      9.80619e-14    -7.17555e-14    44.61           82.4009
 -86.5358       235.595        553.529           -1.68131e-13      4.04388e-13    -2.95905e-13    98.426         187.077
   1.0475e-13    -4.33733e-14   -1.70071e-13   2246.06         -5402.22         3953.0             1.15495e-14    -2.15048e-14
  -6.81608e-14    2.8223e-14     1.10665e-13  -1461.51             4.03353e20     -2.95148e20     -7.51524e-15     1.39931e-14
   4.98757e-14   -2.06518e-14   -8.09777e-14   1069.44            -2.95148e20      2.15971e20      5.49918e-15    -1.02393e-14
  -3.35731       44.61          98.426            9.38819e-15     -2.25804e-14     1.65229e-14  1624.83         -330.993
 -16.9554        82.4009       187.077           -2.23151e-14      5.36721e-14    -3.92739e-14  -330.993       

In [45]:
diag(omega)

8-element Array{Float64,1}:
  183.8290114062524
  625.2318823744095
  553.5294882706805
 2246.0625
    4.0335252084388515e20
    2.159705999841178e20
 1624.8253631926632
 2343.4530257884758

Clustering by year

In [112]:
Years = df.processo[:];

for i in 1:length(Years)
    Years[i] = df.processo[i][12:15]
end

In [118]:
Unique_Years = sort(unique(Years))

10-element Array{Any,1}:
 "2008"
 "2009"
 "2010"
 "2011"
 "2012"
 "2013"
 "2014"
 "2015"
 "2016"
 "2017"

In [168]:
IFCluster = zeros(length(Unique_Years), size(IF,2));

for (t, year) in enumerate(Unique_Years)
    for i in 1:nrow(df)
        if(Years[i]==year)
            IFCluster[t,:] += IF[i, :]
        end
    end
end



In [169]:
W2 = 1/(n) * transpose(IFCluster) * IFCluster

8×8 Array{Float64,2}:
  0.0680611     0.00222196    0.0542794   …  -0.0167901   -0.0216484
  0.00222196    0.126598     -0.0716563       0.0805699    0.0348444
  0.0542794    -0.0716563     0.14343        -0.0810504   -0.0539249
 -0.0146745     0.0251951     0.00493282     -0.00672832  -0.00523013
  6.10489e-19  -7.0966e-19    3.1157e-19      0.0          0.0
  1.22098e-18  -1.41932e-18   6.2314e-19  …   0.0          0.0
 -0.0167901     0.0805699    -0.0810504       0.0733939    0.038145
 -0.0216484     0.0348444    -0.0539249       0.038145     0.0337322

In [171]:
inv(W2)

LoadError: [91mSingularException(8)[39m